In [ ]:
import pandas as pd
from pandas import option_context
import numpy as np

In [ ]:
date_base = '20190930'

In [ ]:
# import 'slownik_tickery.xlsx'
slownik_tickery = pd.read_excel('slownik_tickery_full.xlsx', header = 0)
slownik_tickery

In [ ]:
# import 'slownik_ratings.xlsx'
slownik_ratings = pd.read_excel('slownik_ratings.xlsx', header = 0).drop(['no_ag', 'unikalny'], axis = 1)
slownik_ratings

In [ ]:
# import wsad
pd.set_option('display.max_rows', 200)
wsad = pd.read_excel('wsad_sample.xlsx')
wsad = wsad.dropna(how = 'all')

wsad = wsad.rename(columns = {"Unnamed: 0": 'col1', "Unnamed: 1": 'col2', "Unnamed: 2": 'col3', "Unnamed: 3": 'col4',
                              "Unnamed: 4": 'col5', "Unnamed: 5": 'col6', "Unnamed: 6": 'col7',})
wsad = wsad[wsad.col1 != '#<GO> to View Ratings Profile'] 
wsad = wsad.reset_index()
wsad = wsad.drop(['index'], axis=1)
wsad

In [ ]:
# generate list of indexes for every 'copy-paste' header row
index_list_cp = []
slownik_tickery_BT = slownik_tickery.BloombergTicker
for index, row in wsad.iterrows():
    for ticker in slownik_tickery_BT:
        if type(row['col1']) != float:
            if ticker in row['col1']:
                index_list_cp.append(index)

print(index_list_cp)

In [ ]:
# generate list of agencies
agn = list(np.unique(slownik_ratings.agencja))
agn

In [ ]:
# def generate dataframe
def generate_dataframe(df):
    global Ticker
    for index, row in df.iterrows():
        for ticker in slownik_tickery.BloombergTicker:
            if type(row['col1']) != float:
                if ticker in row['col1']:
                    Ticker = ticker

    # compare SLOWNIK_TICKERY_FULL and WSAD; correction of last line's ratings
    numbers_1 = df['col1']
    numbers_1 = numbers_1.dropna()
    numbers_1 = numbers_1.map(lambda x: x.rstrip(')') if type(x) != float else x)
    numbers_1 = numbers_1.map(lambda x: x.strip() if type(x) != int else x)
    numbers_1 = numbers_1.map(lambda x: None if (type(x) == str and len(x) >= 3) or x in ['', ' '] else int(x))
    numbers_1 = numbers_1.dropna().astype(int)
    max_numbers_1 = max(numbers_1)

    # # if numbers_1 contains more than one value=1,
    # # some ticker is not included in the 'slownik_tickery_full' !!
    # # print warning
    unique, counts = np.unique(numbers_1, return_counts=True)
    dictionary = dict(zip(unique, counts))
    if dictionary[1] > 1:
        print('Porównaj ticker we "WSAD" ze "slownik_tickery_full", kolejny po {}'.format(Ticker))

    # correction of last line's ratings
    if max_numbers_1 != len(numbers_1):
        print(Ticker)
        list_numbers_1 = list(numbers_1)

        for x in range(len(list_numbers_1)):
            if (x + 1) != list_numbers_1[x]:
                # print(list_numbers_1[x])

                df = df.reset_index()

                ind = numbers_1.index[x]

                first_rec_to_assign = df.loc[df['index'] == ind].index.values[0]

                # Is there any agency name above the rating to move ?
                if str(list_numbers_1[x - 1]) not in str(df['col1'].iloc[first_rec_to_assign - 1]):
                    records_to_assign = df.iloc[first_rec_to_assign - 1:]

                    records_to_assign['col4'] = records_to_assign.loc[:, 'col1']
                    records_to_assign['col5'] = records_to_assign.loc[:, 'col2']
                    records_to_assign['col6'] = records_to_assign.loc[:, 'col3']

                    records_to_assign[['col1', 'col2', 'col3']] = np.nan, np.nan, np.nan

                    df = df.iloc[:first_rec_to_assign - 1]
                    df = df.append(records_to_assign)
                    print(records_to_assign)

                else:

                    records_to_assign = df.loc[df['index'] >= ind]
                    records_to_assign['col4'] = records_to_assign.loc[:, 'col1']
                    records_to_assign['col5'] = records_to_assign.loc[:, 'col2']
                    records_to_assign['col6'] = records_to_assign.loc[:, 'col3']

                    records_to_assign[['col1', 'col2', 'col3']] = np.nan, np.nan, np.nan
                    df = df.loc[df['index'] < ind]
                    df = df.append(records_to_assign)
                #                     print(records_to_assign)

                break

    bloom = list(df['col2']) + list(df['col5'])
    rat = list(df['col3']) + list(df['col6'])
    source = pd.DataFrame({'bloom': bloom, 'Rating': rat})

    source['Rating'] = source['Rating'].map(
        lambda z: z.rstrip(' (Continued)') if type(z) == str and '(Continued)' in z else z)
    
    for index, row in source.iterrows():
        for agency in agn:
            if str(row['bloom']) == agency or str(row['Rating']) == agency:
                source.loc[index, 'bloom'] = agency
                source.loc[index, 'Rating'] = agency

    source['Ticker'] = Ticker

    lista_df.append(source)

In [ ]:
# generate dataframe
lista_df = []

length_list_cp = len(index_list_cp)
for i in range(length_list_cp):
    if i < length_list_cp - 1:

        df = wsad.iloc[index_list_cp[i]:index_list_cp[i + 1]]

        generate_dataframe(df=df)


    else:
        df = wsad.iloc[index_list_cp[i]:]

        generate_dataframe(df=df)

ratingi = pd.concat(lista_df)
ratingi = ratingi.reset_index()
ratingi = ratingi.drop(['index'], axis=1)
# ratingi.to_excel("output.xlsx")

# print(ratingi.loc[ratingi['Ticker'] == 'CBK GR Equity'])

In [ ]:
collect_dataframes = []

# divide

for ticker in np.unique(list(slownik_tickery.BloombergTicker)):
    if len(ratingi.loc[ratingi['Ticker'] == ticker]) > 0:
        
        df_ticker = ratingi.loc[ratingi['Ticker'] == ticker].reset_index().drop(['index'], axis=1)
        df_ticker = df_ticker.fillna(' ')
        

        # generate dataframes for each agency
        index_list_agn = []
        
        for index, row in df_ticker.iterrows():
            for agency in agn:
                bloom_str = str(row['bloom'])
                Rating_str = str(row['Rating'])
                
                if bloom_str == agency or Rating_str == agency:
                    index_list_agn.append(index)
            
        
        # how many agencies per ticker ?
        length_index_list_agn = len(index_list_agn)
        
        # create sub dataframe for each agency
        for i in range(length_index_list_agn):
            if i < length_index_list_agn - 1:

                # get next stop
                sub_ticker = df_ticker.iloc[index_list_agn[i]:]
                
                space_index_list = []
                agency_index_list = []
                for index, row in sub_ticker.iterrows():
                    if row['bloom'] in [' ', '', np.NaN] or row['Rating'] in [' ', '', np.NaN]:
                        space_index_list.append(index)
                    elif row['bloom'] in agn or row['Rating'] in agn:
                        agency_index_list.append(index)
                
                for j in space_index_list:
                    if j > index_list_agn[i]:
                        sub_ticker = df_ticker.iloc[index_list_agn[i]:j]
                        sub_ticker['agency'] = df_ticker.loc[index_list_agn[i], 'bloom']
                        merged_sub_ticker = sub_ticker.merge(slownik_tickery, how='left', left_on='Ticker', right_on='BloombergTicker')
                        merged_sub_ticker = merged_sub_ticker.merge(slownik_ratings, how='inner', left_on=['bloom', 'agency'],
                                                                    right_on = ['bloom', 'agencja'])
                        collect_dataframes.append(merged_sub_ticker)

                        break
            
            else:
                # get next stop
                sub_ticker = df_ticker.iloc[index_list_agn[i]:]
                
                space_index_list = []
                agency_index_list = []
                for index, row in sub_ticker.iterrows():
                    if row['bloom'] in [' ', '', np.NaN] or row['Rating'] in [' ', '', np.NaN]:
                        space_index_list.append(index)
                    elif row['bloom'] in agn or row['Rating'] in agn:
                        agency_index_list.append(index)
                
                for j in space_index_list:
                    if j > index_list_agn[i]:
                        sub_ticker = df_ticker.iloc[index_list_agn[i]:j]
                        sub_ticker['agency'] = df_ticker.loc[index_list_agn[i], 'bloom']
                        merged_sub_ticker = sub_ticker.merge(slownik_tickery, how='left', left_on='Ticker', right_on='BloombergTicker')
                        merged_sub_ticker = merged_sub_ticker.merge(slownik_ratings, how='inner', left_on=['bloom', 'agency'],
                                                                    right_on = ['bloom', 'agencja'])
                        collect_dataframes.append(merged_sub_ticker)

                        break
    


In [ ]:
# Create final dataframe
final = pd.concat(collect_dataframes)
final = final.rename(columns={'id':'TypeDict_ID'})
final = final.drop(['agency', 'CompanyDescription', 'Ticker', 'nazwa wprowadzana'], axis=1)
final['Rating'] = final['Rating'].map(lambda x: x.strip())
# final = final.groupby(['BloombergTicker', 'TypeDict_ID']).count().reset_index()
final.to_excel('WYNIKI.xlsx')
print(len(final))
final